<a href="https://colab.research.google.com/github/RamyHamrouni/NLP-notebooks/blob/main/Presentation_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
import numpy as np
import matplotlib.pyplot as plt

In [17]:
%pip install langchain_tavily

# Task
Create a LangGraph orchestrator pattern with dummy tools and run it.

## Install langgraph

### Subtask:
Install the necessary library.


**Reasoning**:
The subtask is to install the `langgraph` library. This can be done using the `pip install` command in a code cell.



In [2]:
%pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 8.1 MB/s eta 0:00:00


In [37]:
from openai import OpenAI
from typing import List, Dict, Any, Optional, Type, TypeVar
from pydantic import BaseModel
import json

T = TypeVar("T", bound=BaseModel)

class LLMClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url
        self.api_key = api_key

    def completion(self, system_prompt: str, user_input: str, tools: List[Dict[str, Any]] = None) -> Any:
        raise NotImplementedError("completion must be implemented by subclasses")


class OpenAIClient(LLMClient):
    def __init__(self, base_url: str, api_key: str, model: str = "gpt-4.1-mini", temperature: float = 0.2):
        super().__init__(base_url, api_key)
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.temperature = temperature

    def completion(
        self,
        system_prompt: str,
        messages: List[Dict[str, str]],
        user_input: str = "",
        tools: Optional[List[Dict[str, Any]]] = None,
    ) -> Any:
        params = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature,
        }

        if tools is not None:
            params["tools"] = tools
            params["tool_choice"] = "auto"

        response = self.client.chat.completions.create(**params)
        return response.choices[0].message.content

    # 🧠 Structured output method — like LangChain’s with_structured_output
    def structured_completion(
        self,
        messages: List[Dict[str, str]],
        schema: Type[T],
        system_prompt: Optional[str] = None,
    ) -> T:
        """
        Generate structured output that conforms to a given Pydantic schema.
        """
        # Add instruction for JSON output with the schema’s structure
        schema_json = json.dumps(schema.model_json_schema(), indent=2)
        structure_hint = (
            "Respond ONLY in valid JSON that fits this schema:\n"
            f"{schema_json}"
        )

        # Insert structure hint into messages
        full_messages = [
            {"role": "system", "content": system_prompt or "You are a helpful assistant."},
            *messages,
            {"role": "system", "content": structure_hint},
        ]

        response = self.client.chat.completions.create(
            model=self.model,
            messages=full_messages,
            temperature=self.temperature,
            response_format={"type": "json_object"},  # ensures JSON-only response
        )

        content = response.choices[0].message.content
        try:
            data = json.loads(content)
            return schema.model_validate(data)
        except Exception as e:
            raise ValueError(f"Failed to parse structured output: {e}\nRaw output: {content}")


In [26]:
class ContextManager:
    def __init__(self,messages:List):
        self.messages = messages

    def add_message(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})

In [31]:
def create_expanded_context(
        base_prompt: str,
        role: Optional[str] = None,
        examples: Optional[List[str]] = None,
        constraints: Optional[List[str]] = None,
        audience: Optional[str] = None,
        tone: Optional[str] = None,
        output_format: Optional[str] = None
    ) -> str:
          """
          Create an expanded context from a base prompt with optional components.

          Args:
              base_prompt: The core instruction or question
              role: Who the model should act as
              examples: List of example outputs to guide the model
              constraints: List of requirements or boundaries
              audience: Who the output is intended for
              tone: Desired tone of the response
              output_format: Specific format requirements

          Returns:
              Expanded context as a string
          """
          context_parts = []

          # Add role if provided
          if role:
              context_parts.append(f"You are {role}.")

          # Add base prompt
          context_parts.append(base_prompt)

          # Add audience if provided
          if audience:
              context_parts.append(f"Your response should be suitable for {audience}.")

          # Add tone if provided
          if tone:
              context_parts.append(f"Use a {tone} tone in your response.")

          # Add output format if provided
          if output_format:
              context_parts.append(f"Format your response as {output_format}.")

          # Add constraints if provided
          if constraints and len(constraints) > 0:
              context_parts.append("Requirements:")
              for constraint in constraints:
                  context_parts.append(f"- {constraint}")

          # Add examples if provided
          if examples and len(examples) > 0:
              context_parts.append("Examples:")
              for i, example in enumerate(examples, 1):
                  context_parts.append(f"Example {i}:\n{example}")

          # Join all parts with appropriate spacing
          expanded_context = "\n\n".join(context_parts)

          return expanded_context

In [23]:
import os
from google.colab import userdata

# Get Tavily API key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

# Get OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [24]:
from langchain_tavily import TavilySearch

web_search = TavilySearch(max_results=3)
web_search_results = web_search.invoke("who is the mayor of NYC?")

print(web_search_results["results"][0]["content"])

[Skip to main content](https://www.nyc.gov/mayors-office/mayors-bio#mainContent) *   [Services](https://www.nyc.gov/main/services) *   [View all services](https://www.nyc.gov/main/services) *   [Events](https://www.nyc.gov/main/events) *   [Office of the Mayor](https://www.nyc.gov/mayors-office) *   [Your government](https://www.nyc.gov/main/your-government) *   [Understanding local government](https://www.nyc.gov/main/your-government) *   [nyc.gov home](https://www.nyc.gov/main) [NYC](https://www.nyc.gov/) [NYC](https://www.nyc.gov/) [Office of the Mayor](https://www.nyc.gov/mayors-office) *   [News](https://www.nyc.gov/mayors-office/news) *   [Contact the Mayor](https://www.nyc.gov/mayors-office/contact-the-mayor) *   [Biography](https://www.nyc.gov/mayors-office/mayors-bio) [Contact the Mayor](https://www.nyc.gov/mayors-office/contact-the-mayor) ![Image 2](https://www.nyc.gov/adobe/dynamicmedia/deliver/dm-aid--052e534c-1b88-4f18-835e-49b66126c154/mayors-bio-crop-jpg.webp?preferwebp=

In [25]:
"""from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-relatd tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="research_agent",
)"""

'from langgraph.prebuilt import create_react_agent\n\nresearch_agent = create_react_agent(\n    model="openai:gpt-4.1-mini",\n    tools=[web_search],\n    prompt=(\n        "You are a research agent.\n\n"\n        "INSTRUCTIONS:\n"\n        "- Assist ONLY with research-relatd tasks, DO NOT do any math\n"\n        "- After you\'re done with your tasks, respond to the supervisor directly\n"\n        "- Respond ONLY with the results of your work, do NOT include ANY other text."\n    ),\n    name="research_agent",\n)'

In [35]:
from pydantic import BaseModel, Field
class CreativeBrief(BaseModel):
      target_audience: str = Field(description="The target audience for the brief")
      primary_goal: str = Field(description="The primary goal of the brief")
      professional_tone: str = Field(description="The professional tone of the brief")
      key_takeaway: str = Field(description="The key takeaway from the brief")


In [39]:

# Define the base system prompt
STRATEGIST_SYSTEM_PROMPT = create_expanded_context(
    base_prompt="""
    Your job is to take a user's topic and create a 'Creative Brief' for the rest of the team.
    You must define:
      - target_audience
      - primary_goal (e.g., 'to inform', 'to persuade', 'to get funding')
      - professional_tone (e.g., 'casual', 'formal', 'academic')
      - key_takeaway

    Return your response as a structured JSON object.
    """,
    role="You are an expert communications strategist and audience analyst.",
)

llm_client=OpenAIClient(
          api_key="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",
          base_url="https://openrouter.ai/api/v1",
          model="gpt-5-mini"
    )
messages = [
    {"role": "system", "content": STRATEGIST_SYSTEM_PROMPT},
    {"role": "user", "content": "Create a creative brief for an AI-driven fitness app that personalizes workouts for beginners."}
]

brief = llm_client.structured_completion(messages, CreativeBrief, system_prompt=STRATEGIST_SYSTEM_PROMPT)

print("🎯 Structured Creative Brief:")
print(brief)

🎯 Structured Creative Brief:
target_audience='Health-conscious beginners (ages 20–50) who are new to structured exercise—busy professionals, parents, and older novices seeking safe, time-efficient, guided workouts; tech-curious but potentially intimidated by gyms and looking for personalized support, accountability, and low injury risk.' primary_goal='To persuade — acquire and retain beginner users by convincing them to download, start a trial, and engage regularly through AI-personalized, safe, and time-efficient workout plans.' professional_tone='Approachable and encouraging' key_takeaway='Use AI to deliver personalized, beginner-friendly workouts that prioritize safety, short time commitments, progressive skill-building, and measurable progress to help users build confidence and sustainable exercise habits.'


In [41]:
brief_dict = brief.model_dump()

In [44]:
brief_dict

{'target_audience': 'Health-conscious beginners (ages 20–50) who are new to structured exercise—busy professionals, parents, and older novices seeking safe, time-efficient, guided workouts; tech-curious but potentially intimidated by gyms and looking for personalized support, accountability, and low injury risk.',
 'primary_goal': 'To persuade — acquire and retain beginner users by convincing them to download, start a trial, and engage regularly through AI-personalized, safe, and time-efficient workout plans.',
 'professional_tone': 'Approachable and encouraging',
 'key_takeaway': 'Use AI to deliver personalized, beginner-friendly workouts that prioritize safety, short time commitments, progressive skill-building, and measurable progress to help users build confidence and sustainable exercise habits.'}

In [4]:
from langgraph.graph import StateGraph, END

# Define the state

class GraphState:
    def __init__(self):
        self.ContextManager = ContextManager(messages=[])
        self.ContextManager.add_message("system", "You are a helpful assistant.")
        self.LLMClient= OpenAIClient(
          api_key="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",
          base_url="https://openrouter.ai/api/v1",
          model="gpt-5-mini"
        )
        self.next = None

# Create an instance of StateGraph
graph = StateGraph(GraphState)

# Define a node to handle tool calls
def call_tool(state: GraphState):
    # In a real scenario, this node would execute the tool calls
    # and update the state with the results.
    print("Calling tool...")
    state.messages.append("Tool called")
    state.next = "end"  # Simple flow: tool calls lead to the end
    return state

# Add the defined nodes to the StateGraph instance
graph.add_node("orchestrator", orchastrato)

# Set the entry point of the graph
graph.set_entry_point("tool_caller")

# Set the finish point of the graph
graph.set_finish_point("tool_caller") # For this simple example, the tool caller leads to the end

# Compile the graph
app = graph.compile()

## Define tools

### Subtask:
Define the tools that the orchestrator will use.


**Reasoning**:
Import the necessary decorator and define the dummy tools as instructed.



In [3]:
from langchain_core.tools import tool

@tool
def tool1(query: str) -> str:
    """Calls tool1 with the given query."""
    return f"Tool1 was called with: {query}"

@tool
def tool2(query: str) -> str:
    """Calls tool2 with the given query."""
    return f"Tool2 was called with: {query}"

tools = [tool1, tool2]

## Define the graph

### Subtask:
Define the LangGraph graph with nodes and edges representing the orchestrator pattern.


**Reasoning**:
Import necessary classes, define the state, create the graph instance, define and add nodes, set the entry and finish points, and compile the graph.



## Run the graph

### Subtask:
Run the defined graph to demonstrate the orchestrator pattern.


**Reasoning**:
Define the initial state and invoke the compiled graph with this state. Then print the result.



In [5]:
# Define the initial state
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd245aca5a0>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error message indicates that the node `call_tool` is returning a `GraphState` object, but it should be returning a dictionary representing the state updates. Modify the `call_tool` function to return a dictionary.



In [6]:
# Define a node to handle tool calls
def call_tool(state: GraphState):
    # In a real scenario, this node would execute the tool calls
    # and update the state with the results.
    print("Calling tool...")
    # Return a dictionary with the updates to the state
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Re-compile the graph
app = graph.compile()

# Define the initial state
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd245b96240>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error persists because the graph is still configured to return the entire `GraphState` object when setting the finish point. The finish point should represent the final state or output, which in this case should be a dictionary reflecting the state after the tool call. Modify the graph definition to explicitly indicate that the output is the state dictionary when setting the finish point.



In [7]:
from langgraph.graph import StateGraph, END

# Define the state - already defined in a previous cell
# class GraphState:
#     def __init__(self):
#         self.messages = []
#         self.tool_calls = None
#         self.next = None

# Create an instance of StateGraph - already created
# graph = StateGraph(GraphState)

# Define a node to handle tool calls - already defined and modified
def call_tool(state: GraphState):
    print("Calling tool...")
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance - already done
# graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph - already done
# graph.set_entry_point("tool_caller")

# Set the finish point of the graph to return the state dictionary
graph.set_finish_point("tool_caller", mapper=lambda x: x.__dict__)

# Compile the graph
app = graph.compile()

# Define the initial state - already done
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

TypeError: StateGraph.set_finish_point() got an unexpected keyword argument 'mapper'

**Reasoning**:
The error indicates that `set_finish_point` does not accept a `mapper` argument. Looking at the LangGraph documentation, the finish point simply indicates the node where the graph execution stops. The final output is typically the state of the graph at that point. The previous error was likely due to the node itself returning an object instead of a dictionary. Since the `call_tool` node is now returning a dictionary, setting the finish point to this node should work correctly.



In [8]:
from langgraph.graph import StateGraph, END

# Define the state - already defined in a previous cell
# class GraphState:
#     def __init__(self):
#         self.messages = []
#         self.tool_calls = None
#         self.next = None

# Create an instance of StateGraph - already created
# graph = StateGraph(GraphState)

# Define a node to handle tool calls - already defined and modified
def call_tool(state: GraphState):
    print("Calling tool...")
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance - already done
# graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph - already done
# graph.set_entry_point("tool_caller")

# Set the finish point of the graph to the tool_caller node
graph.set_finish_point("tool_caller")

# Compile the graph
app = graph.compile()

# Define the initial state - already done
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd2458ce1e0>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error message `InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x...>` indicates that the state update is not in the expected dictionary format. Although the `call_tool` function is returning a dictionary, the state being passed around in the graph is still the `GraphState` object. The `GraphState` class needs to inherit from `TypedDict` to work correctly with LangGraph's state management. Also, ensure that the state class is defined correctly and used consistently. Let's redefine the `GraphState` class inheriting from `TypedDict` and then re-run the graph.



In [9]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, END
import operator

# Define the state as a TypedDict
class GraphState(TypedDict):
    """Represents the state of our graph."""
    messages: Annotated[List[str], operator.add]
    tool_calls: Annotated[List[dict] | None, operator.add]
    next: str | None

# Create an instance of StateGraph
graph = StateGraph(GraphState)

# Define a node to handle tool calls
def call_tool(state: GraphState):
    print("Calling tool...")
    # The node should return updates to the state as a dictionary
    return {"messages": ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance
graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph
graph.set_entry_point("tool_caller")

# Set the finish point of the graph
graph.set_finish_point("tool_caller")

# Compile the graph
app = graph.compile()

# Define the initial state
# The initial state should also be a dictionary conforming to GraphState
initial_state: GraphState = {"messages": [], "tool_calls": None, "next": None}

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

Calling tool...
Final State: {'messages': ['Tool called'], 'tool_calls': None, 'next': 'end'}


## Summary:

### Data Analysis Key Findings

*   The `langgraph` library was successfully installed, including its dependencies.
*   Two dummy tools, `tool1` and `tool2`, were defined using the `@tool` decorator from `langchain_core.tools`.
*   A basic LangGraph `StateGraph` was defined with a single node named `tool_caller` which simulates calling a tool.
*   Initially, running the graph failed with an `InvalidUpdateError` because the `GraphState` was defined as a custom class instead of inheriting from `typing.TypedDict`, which is required for LangGraph's state management.
*   After redefining `GraphState` as a `TypedDict` and providing the initial state as a dictionary, the graph executed successfully.
*   The final state of the graph shows the message "Tool called" was added, indicating the `call_tool` node was executed.

### Insights or Next Steps

*   Future development should focus on integrating actual tool execution within the `call_tool` node and handling the results to update the graph state appropriately.
*   The graph can be expanded to include more complex logic, such as conditional routing based on tool outputs or the addition of an agent node to decide which tool to call.
